
### **Location Chat (Thread-style)**
- `POST /chat/{location_id}` – send message
- `GET /chat/{location_id}` – get latest messages

### **Chat Rooms (Global Rooms)**
- `GET /chat/rooms` – list rooms
- `POST /chat/rooms` – create room
- `GET /chat/rooms/{room_id}/messages` – paginated history
- `POST /chat/rooms/{room_id}/messages` – send message via HTTP fallback

**Note:** WebSocket endpoints are not tested here.


In [ ]:
import requests
import json

BASE_URL = "http://localhost:8000"

# Requires user authentication (same as interactions)
USER_ID = "1c0eb7ac-9a34-43f2-9444-26cc633919c4"

if USER_ID == "PUT-YOUR-USER-ID-HERE":
    print("⚠️ Please set USER_ID to a valid UUID from /auth/login.")
else:
    print("USER_ID set to:", USER_ID)

HEADERS = {"X-User-ID": USER_ID}
print("BASE_URL =", BASE_URL)


USER_ID set to: 1c0eb7ac-9a34-43f2-9444-26cc633919c4
BASE_URL = http://localhost:8000


## 1️⃣ Ensure a Test Location Exists

Location-based chat needs a valid `location_id`.
- Try to fetch existing locations
- If none exist, create one

In [2]:
def get_any_location_id():
    url = f"{BASE_URL}/locations/"
    res = requests.get(url)
    if res.status_code != 200:
        print("GET /locations/ failed", res.status_code)
        return None

    locations = res.json()
    if not locations:
        return None
    return locations[0]["id"]

test_location_id = get_any_location_id()

if not test_location_id:
    print("No locations found → creating test location…")
    payload = {
        "name": "Chat Test Location",
        "description": "Used to test chat endpoints.",
        "maps_url": "https://maps.example.com",
        "price_level": 2,
        "area": "Testing"
    }
    res = requests.post(f"{BASE_URL}/locations/", json=payload)
    test_location_id = res.json().get("id")

print("Using location_id:", test_location_id)
test_location_id

Using location_id: de3763f6-41a4-4370-bb03-0baa7f921ecc


'de3763f6-41a4-4370-bb03-0baa7f921ecc'

## 2️⃣ POST /chat/{location_id} — Send a Message to Location Chat


In [ ]:
url = f"{BASE_URL}/chat/{test_location_id}"
payload = {"message": "Hello from the REST chat tester!"}
res = requests.post(url, json=payload, headers=HEADERS)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


Status: 200
{
  "message": "Hello from the REST chat tester!",
  "id": 1,
  "location_id": "de3763f6-41a4-4370-bb03-0baa7f921ecc",
  "user_id": "1c0eb7ac-9a34-43f2-9444-26cc633919c4",
  "created_at": "2025-11-24T03:12:28.605032+08:00"
}


## 3️⃣ GET /chat/{location_id} — Fetch Messages for Location Chat


In [4]:
url = f"{BASE_URL}/chat/{test_location_id}"
res = requests.get(url, headers=HEADERS)

print("Status:", res.status_code)
print(json.dumps(res.json()[:5], indent=2))  # first 5 messages


Status: 200
[
  {
    "message": "Hello from the REST chat tester!",
    "id": 1,
    "location_id": "de3763f6-41a4-4370-bb03-0baa7f921ecc",
    "user_id": "1c0eb7ac-9a34-43f2-9444-26cc633919c4",
    "created_at": "2025-11-24T03:12:28.605032+08:00"
  }
]


# 🗂️ Chat Rooms API

Now testing global chat room endpoints.

## 4️⃣ GET /chat/rooms — List All Chat Rooms

In [5]:
url = f"{BASE_URL}/chat/rooms"
res = requests.get(url, headers=HEADERS)

print("Status:", res.status_code)
rooms = res.json()
print("Rooms:", json.dumps(rooms, indent=2))


Status: 422
Rooms: {
  "detail": [
    {
      "type": "uuid_parsing",
      "loc": [
        "path",
        "location_id"
      ],
      "msg": "Input should be a valid UUID, invalid character: expected an optional prefix of `urn:uuid:` followed by [0-9a-fA-F-], found `r` at 1",
      "input": "rooms",
      "ctx": {
        "error": "invalid character: expected an optional prefix of `urn:uuid:` followed by [0-9a-fA-F-], found `r` at 1"
      }
    }
  ]
}


## 5️⃣ POST /chat/rooms — Create a New Room

This endpoint allows creating custom chat rooms.

In [6]:
payload = {"name": "Test Room", "category": "testing"}
url = f"{BASE_URL}/chat/rooms"

res = requests.post(url, json=payload, headers=HEADERS)
print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))

room_id = res.json().get("id")
room_id

Status: 422
{
  "detail": [
    {
      "type": "uuid_parsing",
      "loc": [
        "path",
        "location_id"
      ],
      "msg": "Input should be a valid UUID, invalid character: expected an optional prefix of `urn:uuid:` followed by [0-9a-fA-F-], found `r` at 1",
      "input": "rooms",
      "ctx": {
        "error": "invalid character: expected an optional prefix of `urn:uuid:` followed by [0-9a-fA-F-], found `r` at 1"
      }
    },
    {
      "type": "missing",
      "loc": [
        "body",
        "message"
      ],
      "msg": "Field required",
      "input": {
        "name": "Test Room",
        "category": "testing"
      }
    }
  ]
}


## 6️⃣ GET /chat/rooms/{room_id}/messages — Fetch Room History

Uses optional query parameters:
- `limit`
- `before` (ISO8601 timestamp)

In [7]:
url = f"{BASE_URL}/chat/rooms/{room_id}/messages?limit=20"
res = requests.get(url, headers=HEADERS)

print("Status:", res.status_code)
msgs = res.json()
print(json.dumps(msgs, indent=2))


Status: 422
{
  "detail": [
    {
      "type": "uuid_parsing",
      "loc": [
        "path",
        "room_id"
      ],
      "msg": "Input should be a valid UUID, invalid character: expected an optional prefix of `urn:uuid:` followed by [0-9a-fA-F-], found `N` at 1",
      "input": "None",
      "ctx": {
        "error": "invalid character: expected an optional prefix of `urn:uuid:` followed by [0-9a-fA-F-], found `N` at 1"
      }
    }
  ]
}


## 7️⃣ POST /chat/rooms/{room_id}/messages — Send Message via HTTP

Fallback API for users who cannot use WebSockets.

In [ ]:
url = f"{BASE_URL}/chat/rooms/{room_id}/messages"
payload = {"text": "Hello from HTTP room sender!"}

res = requests.post(url, json=payload, headers=HEADERS)
print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


## 8️⃣ Confirm Room History Updated


In [ ]:
url = f"{BASE_URL}/chat/rooms/{room_id}/messages?limit=10"
res = requests.get(url, headers=HEADERS)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


## ✅ Done!

You have now tested all REST chat endpoints:
- Location chat send & fetch
- Chat rooms list/create
- Room messages list
- HTTP fallback message send

WebSockets should be tested using a Python script or WebSocket client.
